# 探索的データ分析(EDA)

In [1]:
# ライブラリのインポート
import pandas as pd
import numpy as np

## データ取得

In [3]:
inputdata = pd.read_csv("s3://choitore-data/inputdata/inputdata.csv")

In [45]:
# 学習データ取得
train_df = pd.read_csv("s3://choitore-ai/Dataset/train1.csv")

In [46]:
# テストデータ取得
test_df = pd.read_csv("s3://choitore-ai/Dataset/test.csv")

## データの概要把握

In [47]:
# データ数確認
# 教師データ
print(train_df.shape)
# テストデータ
print(test_df.shape)

(6000, 14)
(4277, 13)


In [48]:
# データの中身確認
# 学習データ
train_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [49]:
# テストデータ
test_df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [50]:
# 学習データの各カラムのデータタイプ
train_df.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

In [51]:
# 学習データの統計情報
train_df.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,5867.000000,5875.000000,5868.000000,5847.000000,5876.000000,5860.000000
mean,28.888359,227.077106,466.221370,168.529844,309.742342,309.229863
std,14.632432,672.897788,1674.622624,527.917773,1134.165840,1165.112064
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,48.000000,77.250000,24.000000,59.000000,45.250000
max,79.000000,14327.000000,29813.000000,10424.000000,22408.000000,24133.000000


In [52]:
# 学習データの統計情報
test_df.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,4186.000000,4195.000000,4171.000000,4179.000000,4176.000000,4197.000000
mean,28.658146,219.266269,439.484296,177.295525,303.052443,310.710031
std,14.179072,607.011289,1527.663045,560.821123,1117.186015,1246.994742
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,37.000000,53.000000,78.000000,33.000000,50.000000,36.000000
max,79.000000,11567.000000,25273.000000,8292.000000,19844.000000,22272.000000


In [60]:
# 学習データの各カラム毎の欠損値の合計
train_df.isnull().sum()

PassengerId       0
HomePlanet      136
CryoSleep       144
Cabin           144
Destination     130
Age             133
VIP             134
RoomService     125
FoodCourt       132
ShoppingMall    153
Spa             124
VRDeck          140
Name            143
Transported       0
dtype: int64

In [63]:
# テストデータの各カラム毎の欠損値の合計
test_df.isnull().sum()

PassengerId       0
HomePlanet       87
CryoSleep        93
Cabin           100
Destination      92
Age              91
VIP              93
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
Name             94
dtype: int64

In [53]:
# PassengerId の種類ごとの人数
train_df["PassengerId"].value_counts()

PassengerId
6349_01    1
0001_01    1
0002_01    1
0003_01    1
0003_02    1
          ..
0017_01    1
0017_02    1
0020_01    1
0020_02    1
0020_03    1
Name: count, Length: 6000, dtype: int64

In [54]:
# HomePlanetの種類ごとの人数
train_df["HomePlanet"].value_counts()

HomePlanet
Earth     3173
Europa    1453
Mars      1238
Name: count, dtype: int64

In [55]:
# CryoSleepの種類ごとの人数
train_df["CryoSleep"].value_counts()

CryoSleep
False    3758
True     2098
Name: count, dtype: int64

In [56]:
# Cabinの種類ごとの人数
train_df["Cabin"].value_counts()

Cabin
G/734/S     8
B/201/P     7
E/13/S      7
F/1194/P    7
B/82/S      7
           ..
G/1017/P    1
C/234/S     1
E/401/P     1
F/1208/S    1
G/1018/P    1
Name: count, Length: 4519, dtype: int64

In [57]:
# Destinationの種類ごとの人数
train_df["Destination"].value_counts()

Destination
TRAPPIST-1e      4089
55 Cancri e      1251
PSO J318.5-22     530
Name: count, dtype: int64

In [58]:
# VIPの種類ごとの人数
train_df["VIP"].value_counts()

VIP
False    5738
True      128
Name: count, dtype: int64

In [59]:
# Transportedの種類ごとの人数
train_df["Transported"].value_counts()

Transported
True     3046
False    2954
Name: count, dtype: int64

In [61]:
# HomePlanetとTransportedの関係
result = train_df.groupby("HomePlanet")["Transported"].value_counts()
print(result)

HomePlanet  Transported
Earth       False          1809
            True           1364
Europa      True            957
            False           496
Mars        True            649
            False           589
Name: count, dtype: int64


In [62]:
# DestinationとTransportedの関係
result = train_df.groupby("Destination")["Transported"].value_counts()
print(result)

Destination    Transported
55 Cancri e    True            759
               False           492
PSO J318.5-22  True            275
               False           255
TRAPPIST-1e    False          2149
               True           1940
Name: count, dtype: int64


# 前処理の方針(探索的データ分析)

- PassengerId(object)      : GroupNumber毎に、グループの人数に置き換え
- HomePlanet(object)       : OneHot
- CryoSleep(object)        : binary(0/1)で置き換える
- Cabin(object)            : 3つのカラムに分割(deck:OneHot/num:そのまま使用/side:binary化)
- Destination(object)      : OneHot
- Age(float64)             : 3つにグループ化(子供[0-20]/大人[20-60]/老人[60-])
- VIP(object)              : binary(0/1)で置き換える
- RoomService(float64)     : FoodCourt,ShoppingMall,Spa,VRDeckのsumを一つのカラムとする(ServiceTotal)
- FoodCourt(float64)       : RoomService,ShoppingMall,Spa,VRDeckのsumを一つのカラムとする(ServiceTotal)
- ShoppingMall(float64)    : RoomService,FoodCourt,Spa,VRDeckのsumを一つのカラムとする(ServiceTotal)
- Spa(float64)             : RoomService,FoodCourt,ShoppingMall,VRDeckのsumを一つのカラムとする(ServiceTotal)
- VRDeck(float64)          : RoomService,FoodCourt,ShoppingMall,Spaのsumを一つのカラムとする(ServiceTotal)
- Name(object)             : 除外
- Transported(bool)        : binary(0/1)で置き換える
- ServiceTotal(float64)    : [RoomService,FoodCourt,ShoppingMall,Spa,VRDeck]の合計

